In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reform with gradual phase-out starting at 300% FPL
# Keeps everything the same until 300% FPL, then rises linearly:
# - 350% FPL: 8%
# - 400% FPL: 10%
# - 450% FPL: 12%
# - 500% FPL: 14%
# - 550% FPL: 16%
# - 600% FPL: 18%

reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  # New bracket at 350% FPL
  "gov.aca.ptc_phase_out_rate[6].threshold": {
    "2026-01-01.2100-12-31": 3.5
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.08
  },
  # Modified bracket at 400% FPL
  "gov.aca.ptc_phase_out_rate[7].threshold": {
    "2026-01-01.2100-12-31": 4.0
  },
  "gov.aca.ptc_phase_out_rate[7].amount": {
    "2026-01-01.2100-12-31": 0.10
  },
  # New bracket at 450% FPL
  "gov.aca.ptc_phase_out_rate[8].threshold": {
    "2026-01-01.2100-12-31": 4.5
  },
  "gov.aca.ptc_phase_out_rate[8].amount": {
    "2026-01-01.2100-12-31": 0.12
  },
  # New bracket at 500% FPL
  "gov.aca.ptc_phase_out_rate[9].threshold": {
    "2026-01-01.2100-12-31": 5.0
  },
  "gov.aca.ptc_phase_out_rate[9].amount": {
    "2026-01-01.2100-12-31": 0.14
  },
  # New bracket at 550% FPL
  "gov.aca.ptc_phase_out_rate[10].threshold": {
    "2026-01-01.2100-12-31": 5.5
  },
  "gov.aca.ptc_phase_out_rate[10].amount": {
    "2026-01-01.2100-12-31": 0.16
  },
  # New bracket at 600% FPL
  "gov.aca.ptc_phase_out_rate[11].threshold": {
    "2026-01-01.2100-12-31": 6.0
  },
  "gov.aca.ptc_phase_out_rate[11].amount": {
    "2026-01-01.2100-12-31": 0.18
  },
  # Remove income eligibility cap
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")

In [3]:
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
weights = baseline.calculate("household_weight", period=2024)

ValueError: Could not find the parameter gov.aca.ptc_phase_out_rate[7].threshold (failed at ptc_phase_out_rate).

In [ ]:
baseline_aca_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=2026).sum()
baseline_aca_eligible/1e6

In [ ]:
baseline_aca_enrollment = baseline.calculate("takes_up_aca_if_eligible", map_to="person", period=2026).sum()
baseline_aca_enrollment/1e6

In [ ]:
period = 2026

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up_r = reformed.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc_r  = reformed.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt_r = reformed.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up_r == 1) & (aca_ptc_r > 0)

people_with_ptc_takeup_wtd_r = (mask.astype(float) * person_wt_r).sum()

print(f"{people_with_ptc_takeup_wtd_r:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")

In [ ]:
period = 2026
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")

In [ ]:
import numpy as np
from policyengine_us import Simulation

# -------------------------------
# 1. Pull household-level results
# -------------------------------
# ACA PTC (baseline and reform)
ptc_base   = baseline.calculate("aca_ptc", map_to="household", period=2026)
ptc_reform = reformed.calculate("aca_ptc",   map_to="household", period=2026)

# Household weights (same for both sims)
hh_wt      = baseline.calculate("household_weight", map_to="household", period=2026)

# -------------------------------
# 2. Weighted sum of the change
# -------------------------------
weighted_total_change = ptc_reform - ptc_base

# Total budgetary impact in billions
print(f"Total budgetary impact: ${weighted_total_change.sum()/1e9:.2f} billion")